# TripAdvisor Recommender Model Building (under construction)

### Summary

<ul>

<li>Tourist attractions are governed by power law-like distributions</li>
<li> The sparseness, lack of rating variance, and low user participation of this partial TripAdvisor dataset does not lend itself well to a recommender system. Systems with user stickiness (Netflix, Amazon) where user feedback can be rapidly captured would likely be more amenable</li>

</ul>

### Introduction

Having the frustration of having Tripadvisor recommend a lot of temples and inspired by the Netflix prize, I wanted to see if I could build a better recommendation engine.  For the dataset go <a href=""> here</a>.  My only limitation was the processing power of my puny laptop.


<b>Data</b>
<ul>
<li>430k reviews, 200k users, 5k reviewed items at 800MB.</li>
</ul>

<b>Processing</b>
<ul>
<li>Scraped, cleaned, and processed with Python</li>
<li>Stored in SQLite</li>
<li>Visualizations done in R</li>
</ul>

<b>Everything is awesome!</b>

A histogram of the tripadvisor ratings show that it is skewed towards the upper end.  Tourists are very satisfied.  The average rating is a high 4.4/5 stars.  This is in contrast to the <a href="http://www.timelydevelopment.com/demos/NetflixPrize.aspx">Netflix data</a>, where the average rating was 3.5/5 stars and roughly normally distributed.  



### Data Characterization


#### Non-normal distributions govern TripAdvisor reviews

Immediately, we notice that the dataset is extremely sparse (0.03%).  In comparison, in 2006, when Netflix held a competition to improve their recommendation algorithm, the sparsity of that dataset was 1%, a 30x difference.

By plotting the distribution of reviews for attractions, we find that it is governed by a power law-like distribution with a very long tail.  A large number of attractions have a handful of reviews while few attractions have a large number of reviews. 
<img src="figs/reviews_item_histo.png" style="max-height: 400; max-width: 600px;">

One plausible explanation is that when dealing with popularity, there is a positive feedback loop, popular items become more popular.  If you have a week of vacation, you are likely to spend it visiting the most popular attraction than the 50th most popular.  This phenomenon of preferential attachment is widely known under many names such as Zipf's law, Pareto principle, and the Matthew effect.

<img src="figs/reviews_user_histo.png" style="max-height: 400; max-width: 600px;">

We also find that in this dataset, users overwhelmingly only rate a single attraction.  There is noticeable a lack of stickiness; users don't keep rating attractions, it seems to be a one-off endeavour.

The following algorithms are detailed in a  <a href="http://public.research.att.com/~volinsky/netflix/kdd08koren.pdf">paper</a> written by the winners of the Neflix competition.  All models were implemented in python, optimized using stochastic gradient descent with L2 regularization utilizing the <a href="http://sifter.org/~simon/journal/20061211.html">Simon Funk algorithm</a>.  Parameters were optimized using exhaustive grid search.

<img src="figs/ratings_histo.png" style="max-height: 400; max-width: 600px;">

Intuitively, this makes sense.  Movies incur a one time upfront cost for production.  Distribution costs of digital goods are neglible and shelf space isn't an issue so Netflix can house an almost infinite selection. Therefore, a bad movie on Netflix can live in perpetuity.

In contrast, tourist attractions are dynamic. If tourists dislike an attraction, the operators will eventually go out of business; thus, there is a positive selection bias in the attractions listed on TripAdvisor.  One would probably observe a similar effect in industries where the customer is looking for a one-time great experience - amusement parks, safaris, etc. How common are mediocre amusement parks?



### General Framework

Despite the perceived difficulties, I was curious to see the improvements in efficiency that algorithms could introduce.

Similar to the Netflix prize competition, the objective function was to minimize the RMSE.  Models were trained using kfolds cross-validation with 5 folds, and the RMSE was averaged over the five folds.

#### Item-based recommender




#### Simple Model
An estimate of the rating $\hat{r}_{u,i}$, by user $u$ for item $i$ can be modeled with simple biases for both the user $b_{u}$ and the item $b_{i}$:
$$ \hat{r}_{u,i} = \mu + b_{u} + b_{i}, $$

where $\mu$ is the overall average rating.
#### Singular Value Decomposition (SVD)

With SVD, the goal is to uncover latent features that explain the dataset.  Each user $u$ is associated with a vector $p_{u}$ and each item with an items-factor vector $q_{i}$.  The inner product of the vectors is then the estimate of the rating:

$$\hat{r}_{u,i} = q_{i}^{T}p_{u}.$$

#### Improved SVD
Improved SVD is a blend of both the simple model and SVD:

$$ \hat{r}_{u,i} = q_{i}^{T}p_{u} +  \mu + b_{u} + b_{i}.$$

The final cost function to be optimized is as follows:

$$\min_{b_{*}, p_{*}, q_{*}}\sum_{(u,i)\in\kappa}({r}_{u,i} - \mu - b_{u} - b_{i} - q_{i}^{T}p_{u})^2 + \lambda(b_{u}^2 + b_{i}^2 + \lVert q_{i}\rVert^2 + \lVert p_{u} \rVert^2),$$

where $\kappa$ is the set of all the reviews and $r_{u,i}$ is the rating given by user $u$ for item $i$.



### Results

<!-- <img src="figs/model_performance.png" style="max-height: 400; max-width: 600px;"> -->

<table style="width:20%">
  <tr>
    <td>Model</td>
    <td>RMSE</td>
  </tr>
  <tr>
    <td>Global Average</td>
    <td>0.89</td>
  </tr>
  <tr>
    <td>Attraction Average</td>
    <td>0.89</td>
  </tr>
</table>

We find that SVD model fares the best with a modest 3% improvement over recommending the average.  Interestingly, the blend of baseline predictors and SVD did not fare as well as either of the individual predictors due to possible overfitting.

However, it's debatable whether further model optimization is worthwhile.  Given that the 97% of users rate < 5 items and the previously mentioned difficulties of the dataset, there is the inevitably of diminishing returns to effort.  The squeezing of water from stone problem.  From a product perspective it may be more productive in implementing better features.

Incorporation of temporal bias.

Next step: incorporating information from the reviews into the prediction using LDA as seen <a href="http://www.yelp.com/html/pdf/YelpDatasetChallengeWinner_HiddenFactors.pdf">here</a>.

### Conclusions

After reviewing the collected TripAdvisor dataset, there seems to be limited gains from implementing a recommender system based purely on ratings. 

However, the dataset is very rich in reviews and can be mined for insights into idiosyncratic characteristics of tourist attractions.  To that end, a better product involving this dataset may be a search function designed to look for locations that aren't categorized into preset TripAdvisor categories.